In [0]:
# #project 

# 1. print the total number of employees in the company
# 2. print the total number of department names in the company
# 3.print the department  names of the company
# 4 print the total numbet of emploees in the company
# 5 print the total numbet of emploees in each state
# 6 print the total numbers of employees in the each department
# 7 print the maximum and minimum number of salaries and sort them in ascending order
# 9 print the name of all employyes working in newyork under finanace departmen whose bonuses are greater than average bonus
# 9 raise the salary of all employees by $500 whose age is greater than 45 and save them in file

In [0]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col,lit,udf
from pyspark.sql.types import IntegerType,DoubleType
from pyspark.sql.functions import sum,avg,max,min,mean,count
spark=SparkSession.builder.appName('Spark dataframe project').getOrCreate()

In [0]:
df=spark.read.options(header='True',inferSchema='True').csv('/FileStore/tables/OfficeDataProject.csv')

In [0]:
df.show()

+-----------+-------------------+----------+-----+------+---+-----+
employee_id| employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
 1000| Nitz Leif| Marketing| CA| 6131| 26| 543|
 1001| Melissia Dedman| Finance| AK| 4027| 43| 1290|
 1002| Rudolph Barringer| HR| LA| 3122| 43| 1445|
 1003| Tamra Amber| Accounts| AK| 5717| 47| 1291|
 1004| Mullan Nitz|Purchasing| CA| 5685| 34| 1394|
 1005| Zollner Karie| Accounts| CA| 2843| 27| 1078|
 1006|Kaczorowski Zollner| Sales| CA| 7201| 21| 1834|
 1007| Nakano Locust| Marketing| LA| 3444| 23| 1823|
 1008| Recalde Kensinger| Accounts| LA| 3704| 48| 1330|
 1009| Imai Hallie| Accounts| AK| 5061| 38| 1557|
 1010| Debroah Gallman| Accounts| NY| 9308| 35| 817|
 1011| Barringer Escoto|Purchasing| WA| 1685| 49| 1706|
 1012| Soules Coogan| Accounts| AK| 8330| 43| 1914|
 1013| Luisa Suzanne| Accounts| CA| 1151| 37| 1095|
 1014| Marvis Cobian|Purchasing| NY| 5061| 41| 1765|
 1015| Cobian Kensinger| Sales| LA| 1983| 21| 632|
 1016| Gilma Margret| Marketing| CA| 2919| 45| 1762|
 1017| Ellingsworth Ilana| Accounts| WA| 9614| 26| 1964|
 1018| Vankirk Jacquelyne|Purchasing| NY| 8636| 47| 1192|
 1019| Zollner Juliana| HR| NY| 9739| 30| 1119|
+-----------+-------------------+----------+-----+------+---+-----+
only showing top 20 rows

In [0]:
# 1. print the total number of employees in the company
df.count()

Out[5]: 1000

In [0]:
# 2. print the total number of department in the company
df.select('department').dropDuplicates(['department']).count()
#this can be also done as
df.groupBy('department').count().show()

+----------+-----+
department|count|
+----------+-----+
 Sales| 169|
 HR| 171|
 Finance| 162|
Purchasing| 166|
 Marketing| 170|
 Accounts| 162|
+----------+-----+

In [0]:
# 3.print the department  names of the company
df.select('department').dropDuplicates(['department']).show()

+----------+
department|
+----------+
 Sales|
 HR|
 Finance|
Purchasing|
 Marketing|
 Accounts|
+----------+

In [0]:
# 4 print the total numbet of emploees in the company
df.groupBy('department').count().show()

+----------+-----+
department|count|
+----------+-----+
 Sales| 169|
 HR| 171|
 Finance| 162|
Purchasing| 166|
 Marketing| 170|
 Accounts| 162|
+----------+-----+

In [0]:
# 5 print the total numbet of emploees in each state
df.groupBy('state').count().show()

+-----+-----+
state|count|
+-----+-----+
 LA| 205|
 CA| 205|
 WA| 208|
 NY| 173|
 AK| 209|
+-----+-----+

In [0]:
df.groupBy('state','department').count().show()

+-----+----------+-----+
state|department|count|
+-----+----------+-----+
 CA| Sales| 42|
 CA| Marketing| 33|
 NY| Accounts| 34|
 NY| Sales| 27|
 CA| Finance| 35|
 CA| Accounts| 35|
 CA|Purchasing| 32|
 WA| HR| 47|
 AK|Purchasing| 30|
 WA| Accounts| 27|
 WA|Purchasing| 38|
 AK| Sales| 38|
 AK| Accounts| 37|
 WA| Marketing| 39|
 LA| HR| 41|
 LA| Sales| 35|
 AK| HR| 25|
 LA| Finance| 29|
 AK| Finance| 37|
 LA|Purchasing| 45|
+-----+----------+-----+
only showing top 20 rows

In [0]:
# 7 print the maximum and minimum number of salaries and sort them in ascending order
df.groupBy('department').agg(min('salary'),max('salary')).orderBy(col('min(salary)').asc()) .show()

+----------+-----------+-----------+
department|min(salary)|max(salary)|
+----------+-----------+-----------+
 Finance| 1006| 9899|
 Accounts| 1007| 9890|
 HR| 1013| 9982|
 Marketing| 1031| 9974|
 Sales| 1103| 9982|
Purchasing| 1105| 9985|
+----------+-----------+-----------+

In [0]:
# 9 print the name of all employyes working in newyork under finanace departmen whose bonuses are greater than average bonus
avg_bonus=df.filter(df.state=='NY').groupBy('state').agg(avg('bonus').alias('average_bonus')).select('average_bonus').collect()[0]['average_bonus']

In [0]:
df.filter((df.state=='NY')&(df.department=='Finance')&(df.bonus>avg_bonus)).show()

+-----------+--------------------+----------+-----+------+---+-----+
employee_id| employee_name|department|state|salary|age|bonus|
+-----------+--------------------+----------+-----+------+---+-----+
 1035| Vivan Sifford| Finance| NY| 1129| 35| 1261|
 1073| Herder Gallman| Finance| NY| 1988| 31| 1402|
 1082| Nena Rocha| Finance| NY| 3417| 25| 1647|
 1087| Leif Lemaster| Finance| NY| 8642| 45| 1782|
 1100|Ellingsworth Meli...| Finance| NY| 7845| 32| 1358|
 1127| Escoto Gilma| Finance| NY| 3426| 41| 1285|
 1161| Georgeanna Laub| Finance| NY| 2469| 26| 1679|
 1175| Durio Tenenbaum| Finance| NY| 2253| 42| 1684|
 1180| Juliana Grigg| Finance| NY| 8178| 42| 1617|
 1215| Tiffani Benz| Finance| NY| 1665| 41| 1969|
 1220| Nitz Ilana| Finance| NY| 2443| 50| 1342|
 1342| Phylicia Antonina| Finance| NY| 8626| 30| 1857|
 1414| Durio Janey| Finance| NY| 2992| 46| 1733|
 1660| Melissia Jere| Finance| NY| 9201| 21| 1533|
 1737| Yukiko Kreamer| Finance| NY| 7396| 20| 1332|
 1738| Nena Kensinger| Finance| NY| 8534| 41| 1610|
 1815| Antonina Ilana| Finance| NY| 2944| 28| 1718|
+-----------+--------------------+----------+-----+------+---+-----+

In [0]:
# 9 raise the salary of all employees by $500 whose age is greater than 45 and save them in file
def increment_sal(age,currentsalary):
  if age > 45:
    return currentsalary + 500
  return currentsalary


increse_salaryUDF = udf(lambda x,y :increment_sal(x,y), IntegerType())


df.withColumn('salary',increse_salaryUDF(col('age'),col('salary'))).show()

+-----------+-------------------+----------+-----+------+---+-----+
employee_id| employee_name|department|state|salary|age|bonus|
+-----------+-------------------+----------+-----+------+---+-----+
 1000| Nitz Leif| Marketing| CA| 6131| 26| 543|
 1001| Melissia Dedman| Finance| AK| 4027| 43| 1290|
 1002| Rudolph Barringer| HR| LA| 3122| 43| 1445|
 1003| Tamra Amber| Accounts| AK| 6217| 47| 1291|
 1004| Mullan Nitz|Purchasing| CA| 5685| 34| 1394|
 1005| Zollner Karie| Accounts| CA| 2843| 27| 1078|
 1006|Kaczorowski Zollner| Sales| CA| 7201| 21| 1834|
 1007| Nakano Locust| Marketing| LA| 3444| 23| 1823|
 1008| Recalde Kensinger| Accounts| LA| 4204| 48| 1330|
 1009| Imai Hallie| Accounts| AK| 5061| 38| 1557|
 1010| Debroah Gallman| Accounts| NY| 9308| 35| 817|
 1011| Barringer Escoto|Purchasing| WA| 2185| 49| 1706|
 1012| Soules Coogan| Accounts| AK| 8330| 43| 1914|
 1013| Luisa Suzanne| Accounts| CA| 1151| 37| 1095|
 1014| Marvis Cobian|Purchasing| NY| 5061| 41| 1765|
 1015| Cobian Kensinger| Sales| LA| 1983| 21| 632|
 1016| Gilma Margret| Marketing| CA| 2919| 45| 1762|
 1017| Ellingsworth Ilana| Accounts| WA| 9614| 26| 1964|
 1018| Vankirk Jacquelyne|Purchasing| NY| 9136| 47| 1192|
 1019| Zollner Juliana| HR| NY| 9739| 30| 1119|
+-----------+-------------------+----------+-----+------+---+-----+
only showing top 20 rows